This notebook serves as grounds to our claims of statistical significance regarding model performance, measured by Root-CADD.

In [16]:
import pandas as pd
import numpy as np
from math import sqrt
from scipy.stats import chi2, wilcoxon
from statsmodels.stats.multitest import multipletests

In [17]:
columns = ["Event log", "Mean", "Best Dist.", "Prophet", "LSTM", "Chronos", "XGBoost", "NPP", "AT-KDE"]

data = [
    ["BPIC12", 4.83, 5.12, 4.46, 4.18, 5.48, 4.26, 5.08, 3.71],
    ["BPIC12CW", 6.61, 6.84, 4.47, 6.27, 8.33, 8.69, 6.79, 3.21],
    ["BPIC12O", 14.75, 14.55, 13.61, 15.00, 11.72, 12.73, 14.59, 11.56],
    ["BPIC12W", 4.78, 4.76, 5.07, 4.54, 4.87, 3.66, 4.48, 4.80],
    ["BPIC13C", 66.22, 66.65, 13.99, 27.93, 14.14, 13.39, 68.70, 13.25],
    ["BPIC17W", 15.63, 15.47, 9.65, 13.74, 13.29, 11.96, 15.57, 10.24],
    ["BPIC19", 300.66, 295.53, 17.83, None, 22.54, 21.38, None, 17.58],
    ["BPIC20D", 10.33, 11.55, 5.95, 6.94, 8.58, 6.90, 11.31, 6.84],
    ["BPIC20I", 28.02, 29.09, 17.41, 15.01, 19.93, 17.37, None, 16.59],
    ["BPIC20P", 25.06, 24.47, 15.08, 10.91, 13.58, 12.58, 25.14, 13.82],
    ["Env.permit", 33.48, 31.99, 17.97, 19.62, 18.19, 14.98, 31.76, 13.74],
    ["HelpDesk", 55.69, 56.15, 23.28, 19.58, 44.39, 36.79, 55.61, 19.48],
    ["Hospital", 22.59, 22.45, 15.89, 22.91, 31.10, 22.25, 22.56, 22.75],
    ["Sepsis", 20.97, 21.55, 15.60, 18.33, 18.19, 18.52, 20.86, 15.56],
    ["P2P", 25.28, 25.91, 22.65, 26.41, 24.42, 20.43, 13.05, 12.87],
    ["CVS", 8.97, 9.02, 6.97, 8.01, 5.78, 6.99, 9.09, 3.54],
    ["Conf. 1000", 12.54, 12.25, 6.26, 7.16, 6.05, 9.68, 12.16, 5.20],
    ["Conf. 2000", 18.23, 17.97, 9.86, 11.33, 10.15, 8.99, 18.30, 6.49],
    ["ACR", 9.48, 9.63, 6.80, 6.49, 8.63, 7.34, 8.78, 7.00],
    ["Production", 9.06, 8.47, 10.66, 7.06, 6.38, 4.15, 8.97, 6.42],
]

df = pd.DataFrame(data, columns=columns)

In [18]:
df

,Event log,Mean,Best Dist.,Prophet,LSTM,Chronos,XGBoost,NPP,AT-KDE
0,BPIC12,4.83,5.12,4.46,4.18,5.48,4.26,5.08,3.71
1,BPIC12CW,6.61,6.84,4.47,6.27,8.33,8.69,6.79,3.21
2,BPIC12O,14.75,14.55,13.61,15.00,11.72,12.73,14.59,11.56
3,BPIC12W,4.78,4.76,5.07,4.54,4.87,3.66,4.48,4.80
4,BPIC13C,66.22,66.65,13.99,27.93,14.14,13.39,68.70,13.25
5,BPIC17W,15.63,15.47,9.65,13.74,13.29,11.96,15.57,10.24
6,BPIC19,300.66,295.53,17.83,NaN,22.54,21.38,NaN,17.58
7,BPIC20D,10.33,11.55,5.95,6.94,8.58,6.90,11.31,6.84
8,BPIC20I,28.02,29.09,17.41,15.01,19.93,17.37,NaN,16.59
9,BPIC20P,25.06,24.47,15.08,10.91,13.58,12.58,25.14,13.82


Perform the Skillings-Mack test instead of Friedman giving missing values (3/160). We will run this via R.

In [19]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [20]:
%R -i df

Run Test:

* H0: Model performances are equal
* H1: !H0

In [21]:
%%R
library(Skillings.Mack)
X <- as.matrix(df[ , -1])   # 20 x 8  (rows = logs, cols = models)
X_t <- t(X)                 # 8 x 20  (rows = models, cols = logs)
res <- Ski.Mack(X_t)

print(res)


Skillings-Mack Statistic =  63.619258 , p-value =  0.000000 
Note: the p-value is based on the chi-squared distribution with d.f. =  7 

$Nblocks
[1] 20

$Ntreatments
[1] 8

$rawdata
              0    1     2    3     4     5      6     7     8     9    10
Mean       4.83 6.61 14.75 4.78 66.22 15.63 300.66 10.33 28.02 25.06 33.48
Best Dist. 5.12 6.84 14.55 4.76 66.65 15.47 295.53 11.55 29.09 24.47 31.99
Prophet    4.46 4.47 13.61 5.07 13.99  9.65  17.83  5.95 17.41 15.08 17.97
LSTM       4.18 6.27 15.00 4.54 27.93 13.74    NaN  6.94 15.01 10.91 19.62
Chronos    5.48 8.33 11.72 4.87 14.14 13.29  22.54  8.58 19.93 13.58 18.19
XGBoost    4.26 8.69 12.73 3.66 13.39 11.96  21.38  6.90 17.37 12.58 14.98
NPP        5.08 6.79 14.59 4.48 68.70 15.57    NaN 11.31   NaN 25.14 31.76
AT-KDE     3.71 3.21 11.56 4.80 13.25 10.24  17.58  6.84 16.59 13.82 13.74
              11    12    13    14   15    16    17   18    19
Mean       55.69 22.59 20.97 25.28 8.97 12.54 18.23 9.48  9.06
Best Dist. 56.1

Given that p < 1%, we are able to reject H0 under high statistical significance. 

Onto pairwise comparisons via Wilcoxon-Signed-Rank testing. 

In [22]:
df

,Event log,Mean,Best Dist.,Prophet,LSTM,Chronos,XGBoost,NPP,AT-KDE
0,BPIC12,4.83,5.12,4.46,4.18,5.48,4.26,5.08,3.71
1,BPIC12CW,6.61,6.84,4.47,6.27,8.33,8.69,6.79,3.21
2,BPIC12O,14.75,14.55,13.61,15.00,11.72,12.73,14.59,11.56
3,BPIC12W,4.78,4.76,5.07,4.54,4.87,3.66,4.48,4.80
4,BPIC13C,66.22,66.65,13.99,27.93,14.14,13.39,68.70,13.25
5,BPIC17W,15.63,15.47,9.65,13.74,13.29,11.96,15.57,10.24
6,BPIC19,300.66,295.53,17.83,NaN,22.54,21.38,NaN,17.58
7,BPIC20D,10.33,11.55,5.95,6.94,8.58,6.90,11.31,6.84
8,BPIC20I,28.02,29.09,17.41,15.01,19.93,17.37,NaN,16.59
9,BPIC20P,25.06,24.47,15.08,10.91,13.58,12.58,25.14,13.82


In [23]:
df_wide = df.set_index("Event log")  
control = "AT-KDE"
others = [c for c in df_wide.columns if c != control]

raw_p = []
names = []
ns = []

for m in others:
    pair = df_wide[[control, m]].dropna()  # remove missing data comparisons
    x = pair[control].to_numpy()  # AT-KDE
    y = pair[m].to_numpy()        # competitor
    
    res = wilcoxon(x, y, alternative="less") # lower CADD is better -> H1: our model < competitor
    raw_p.append(res.pvalue)
    names.append(m)
    ns.append(len(pair))

# Holm–Bonferroni correction
rej, p_holm, _, _ = multipletests(raw_p, alpha=0.01, method="holm") #high significance threshold: 1%

for name, n_used, p0, ph, r in zip(names, ns, raw_p, p_holm, rej):
    print(f"{name:12s}  n={n_used:2d}  p={p0:.3g}  p_holm={ph:.3g}  sig={r}")

Mean          n=20  p=4.77e-06  p_holm=3.34e-05  sig=True
Best Dist.    n=20  p=4.77e-06  p_holm=3.34e-05  sig=True
Prophet       n=20  p=0.00365  p_holm=0.00837  sig=True
LSTM          n=19  p=0.00309  p_holm=0.00837  sig=True
Chronos       n=20  p=9.54e-06  p_holm=4.77e-05  sig=True
XGBoost       n=20  p=0.00279  p_holm=0.00837  sig=True
NPP           n=18  p=3.81e-05  p_holm=0.000153  sig=True
